In [1]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 MB 8.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 13.9 MB/s eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 34.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.8 MB/s eta 0:00:0000:01
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 41.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 34.2 MB/s eta 0:00:0000:0100:01
  Using cached opt_einsum

In [3]:
!pip install opencv-python-headless 
!pip install pillow  

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 23.5 MB/s eta 0:00:0000:0100:01


In [4]:
from tensorflow.keras.applications import ResNet50

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

2024-01-22 13:53:00.371315: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/sn/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


94765736/94765736 [==============================] - 2s 0us/step


In [93]:
import numpy as np
import cv2
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical

class CartridgeDataset(Sequence):
    def __init__(self, data, base_path, batch_size=1, image_size=(512, 512), shuffle=True):
        self.image_info = data['images']
        self.annotations = {ann['image_id']: ann for ann in data['annotations']}
        self.base_path = base_path
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.image_info))
        if self.shuffle:
            np.random.shuffle(self.indexes)
        unique_classes = set([img['id'] for img in image_info])
        print("Unique classes in dataset:", unique_classes)
        num_classes = len(unique_classes)
        print("Number of unique classes:", num_classes)

    def __len__(self):
        return len(self.image_info) // self.batch_size

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = [], []

        for i in batch_indexes:
            image_data = self.image_info[i]
            image_path = f"{self.base_path}/{image_data['file_name']}"

            # Print the image path for debugging
            print(f"Attempting to load image from path: {image_path}")

            image = cv2.imread(image_path)

            # Check if image is loaded successfully
            if image is None:
                raise FileNotFoundError(f"Unable to load image at path: {image_path}")

            image = cv2.resize(image, self.image_size)
            X.append(image)

            # Retrieve annotation for the current image
            annotation = self.annotations.get(image_data['id'])
            if annotation is not None:
                mask = self.create_mask(image_data, annotation)
            else:
                # Create an empty mask if no annotation is found
                mask = np.zeros(self.image_size, dtype=np.uint8)

            y.append(mask)
            class_id = image_data['id']
            # One-hot encode the label
            label = to_categorical(class_id, num_classes=4)  # Adjust num_classes as needed
            y.append(label)

        # Convert lists to numpy arrays
        X = np.array(X, dtype=np.float32) / 255.0  # Normalize image data to [0,1]
        y = np.array(y, dtype=np.uint8)
            # Add a channel dimension to y if necessary
        if y.ndim == 3:
            y = np.expand_dims(y, axis=-1)

        return X, y


    def create_mask(self, image_data, annotation):
        mask = np.zeros((image_data['height'], image_data['width']), dtype=np.uint8)

        for seg in annotation['segmentation']:
            # Reshape segmentation data if necessary
            if isinstance(seg, list) and len(seg) % 2 == 0:
                points = np.array(seg, dtype=np.int32).reshape(-1, 2)
            else:
                # Handle other formats or log an error
                print("Invalid segmentation format:", seg)
                continue

            cv2.fillPoly(mask, [points], 1)

        mask = cv2.resize(mask, self.image_size)
        return mask



In [94]:
# !pip install opencv-python-headless

In [95]:
import json
import os

# Load the annotation data
annotation_file = '9mm_annotation.json' 
with open(annotation_file, 'r') as file:
    data = json.load(file)

image_path = '/Users/sn/Documents/rnn/9mm data/masked'
dataset = CartridgeDataset(data, image_path)


Unique classes in dataset: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91}
Number of unique classes: 91


In [96]:
type(dataset)

__main__.CartridgeDataset

In [97]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)


num_classes = 4  
predictions = Dense(num_classes, activation='softmax')(x)

# This is the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [98]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',  
              metrics=['accuracy'])

In [99]:
from sklearn.model_selection import train_test_split

image_info = data['images']
annotations = data['annotations']

train_images, val_images = train_test_split(image_info, test_size=0.2, random_state=42)  # Adjust the test_size as needed

def filter_annotations(annotations, image_ids):
    return [ann for ann in annotations if ann['image_id'] in image_ids]

train_image_ids = [img['id'] for img in train_images]
val_image_ids = [img['id'] for img in val_images]

train_annotations = filter_annotations(annotations, train_image_ids)
val_annotations = filter_annotations(annotations, val_image_ids)

train_dataset = CartridgeDataset({'images': train_images, 'annotations': train_annotations}, image_path)
val_dataset = CartridgeDataset({'images': val_images, 'annotations': val_annotations}, image_path)


Unique classes in dataset: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91}
Number of unique classes: 91
Unique classes in dataset: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91}
Number of unique classes: 91


In [101]:
print(train_dataset)

In [102]:
model.fit(train_dataset, validation_data=val_dataset, epochs=5)  # Adjust the number of epochs as needed

Attempting to load image from path: /Users/sn/Documents/rnn/9mm data/masked/CWRBF1050.png


IndexError: index 53 is out of bounds for axis 1 with size 4